In [1]:
%pip install boto3==1.34.93 ipython --quiet

Note: you may need to restart the kernel to use updated packages.


In [10]:
import boto3
import json, re
%time
session = boto3.session.Session(profile_name='bedrock');
bedrock_client = session.client(
    service_name='bedrock-runtime',
    # region_name='eu-west-3',
    region_name='us-east-1',
)

# Model id and Claude config
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
claude_config = {
    'max_tokens': 4096,
    'temperature': 0, 
    'anthropic_version': '',
    'stop_sequences': ['Human:']
}

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


In [11]:
# Fill with your json sample files here
cdc_json_file_path = "./src/samples/cdc.json"
cv_json_file_path = "./src/samples/cv.json"
rec_json_file_path = "./src/samples/rec.json"

# Open the JSON file and load its contents
with open(cdc_json_file_path, "r") as file:
    cdc_json_data = json.load(file)
with open(cv_json_file_path, "r") as file:
    cv_json_data = json.load(file)
with open(rec_json_file_path, "r") as file:
    rec_json_data = json.load(file)

# Now you can work with the loaded JSON data
# print(cdc_json_data)
# print(cv_json_data)
# print(rec_json_data)

# Add you json inputs here
rfpRequirements = cdc_json_data
resumeData = cv_json_data

criteria_list = rec_json_data['sous_criteres_techniques_et_ponderation']
experience_list = cv_json_data['experiences_entreprise']
stack_technique_list = cdc_json_data['stack_technique']

In [12]:
# We take only the string part
criteria_list_string = [criteria['critere'][0] for criteria in criteria_list]
print(criteria_list_string)

experience_list_string = [experience.get('stack_technique', experience.get('role')) for experience in experience_list]
print(experience_list_string)
tache_list_string = [taches['tache'][0] if isinstance(taches['tache'], list) else taches['tache'] for experience in experience_list_string for taches in experience]
print(tache_list_string)

stack_technique_list_string = [taches['tache'][0] for taches in stack_technique_list]
print(stack_technique_list_string)

['Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)', 'Nombre et pertinence des expériences similaires', 'Compétences techniques', 'Complexité des missions gérées', 'Connaissance du domaine fonctionnel (Facility management)']
[[{'tache': 'N/A'}], [{'tache': 'Power BI, Oracle 19'}], [{'tache': 'Windev, Windev Mobile, Webdev, SQLServer, HyperFile'}], [{'tache': 'Visual Basic et SQLServer'}], [{'tache': 'Prologue Software, WINDOWS, UNIX, BAL, ABAL, CRITERIA SI/MC'}], [{'tache': 'Prologue Software, BAL, ABAL, CRITERIA SI/MC'}], [{'tache': ['Direction de projet (CODIR, COPROJ)', 3]}, {'tache': ['Participation aux ateliers de cadrage', 3]}, {'tache': ['Accompagnement lors des ateliers de formation', 3]}, {'tache': ['Participation aux ateliers de recette', 3]}], [{'tache': ['Direction de projets sur le produit ERP Ulis.', 3]}, {'tache': ["Suivi de comptes clients de l'ERP Ulis afin de répondre à leurs besoins et organiser le service à fournir.", 3

In [13]:
prompt = f"""
Your are a human resources and procurement specialist.
You will be provided with two json formatted inputs. Please read them carefully, because I'm going to ask you an analysis based on their content.
Here are the two inputs:
1) A list of technical requirements from a request for proposal in json format, contained within REQUIREMENTS
tags
2) An extract of a candidate's resume, in json format, contained within RESUME tags

<REQUIREMENTS>
{rfpRequirements}
</REQUIREMENTS>

<RESUMEDATA>
{resumeData}
</RESUMEDATA>

Your task is to evaluate how well the candidate's skills, experiences and seniority match the
technical requirements, and provide a match score from 0 to 5 along with a justification.

Here are the steps to follow:

1. Read the requirements carefully:
- Look for key technical skills, tools, methodologies and experience levels required
- Make note of any specific requirements related to seniority, years of experience, certifications
etc.
- Make note of the main taches_et_responsabilites, this is what the candidate will be doing on the job

2. Read the candidate's resume carefully:
- Identify the candidate's technical skills, tools and methodologies they are familiar with
- Note their years of experience, seniority level, certifications and other qualifications

3. Compare the requirements and resume:
- For each key requirement, determine if the candidate has a matching skill/experience mentioned in
their resume
- Make a list of requirements that are matched and unmatched based on the resume details
4. For each identified skill/experience from the resume that matches a requirement, write it down
  inside an XML tag as follows:
  <Matched Skill/Experience>
  The identified skill/experience from the resume
  </Matched Skill/Experience>
  <Justification>
  A brief explanation of how this skill/experience matches the corresponding requirement(s)
  </Justification>

5. Determine the match score:
- If all key requirements are matched, the score is 5
- If most key requirements are matched, the score is 4
- If about half the requirements are matched, the score is 3
- If few requirements are matched, the score is 2
- If very few or no requirements are matched, the score is 1 or 0

6. Justify the match score:
<Justification>
- Provide specific examples from the requirements and resume to support why certain requirements
were considered matched or unmatched
- Explain your reasoning for the match score, referring to the proportion of requirements matched
and their importance
</Justification>

7. Output your evaluation:
<Match Score>SCORE</Match Score>
JUSTIFICATION

Make sure to replace REQUIREMENTS with the actual requirements text, RESUME with the resume
text, SCORE with the 0-5 match score number, and JUSTIFICATION with your justification text
following the format specified.

If any of the inputs is missing or you cannot determine a score, simply output: "Unable to evaluate
due to missing information."

Before answering the question, please think about it step-by-step within <thinking></thinking> tags. Then, provide your final answer within <answer></answer> tags.

"""

In [14]:
def get_criteria_prompt(criteria):
  return f"""
Your are a human resources and procurement specialist.
You will be provided with one criteria and two json formatted inputs. Please read them carefully, because I'm going to ask you an analysis based on their content.
Here are the three inputs:
1) A criteria on which the analysis will be based, contained within CRITERIA tags
2) A list of technical requirements from a request for proposal in json format, contained within REQUIREMENTS
tags
3) An extract of a candidate's resume, in json format, contained within RESUME tags

<CRITERIA>
{criteria}
</CRITERIA>
<REQUIREMENTS>
{rfpRequirements}
</REQUIREMENTS>

<RESUMEDATA>
{resumeData}
</RESUMEDATA>

Your task is to evaluate how well the candidate's skills, experiences and seniority match the
criteria, and provide a match score from 0 to 5 along with a justification.
The note should reflect how well the candidate matches the provided criteria.

Here are the steps to follow:

1. Read the requirements carefully with the criteria in mind:
- Look for key technical skills, tools, methodologies and experience levels required that match the criteria
- Make note of any specific requirements related to seniority, years of experience, certifications
etc. that match the criteria
- Make note of the main taches_et_responsabilites that match the criteria, this is what the candidate will be doing on the job

2. Read the candidate's resume carefully with the criteria in mind:
- Identify the candidate's technical skills, tools and methodologies they are familiar with
- Note their years of experience, seniority level, certifications and other qualifications

3. Compare the requirements and resume with the criteria in mind:
- For each key requirement and based on the criteria, determine if the candidate has a matching skill/experience mentioned in
their resume
- Make a list of requirements that are matched and unmatched based on the resume details
4. For each identified skill/experience from the resume that matches a requirement, write it down
  inside an XML tag as follows:
  <MatchedSkillExperience>
  The identified skill/experience from the resume
  </MatchedSkillExperience>
  <Justification>
  A brief explanation of how this skill/experience matches the corresponding requirement(s) based on the criteria
  </Justification>

5. Determine the match score:
- If all key requirements are matched, the score is 5
- If most key requirements are matched, the score is 4
- If about half the requirements are matched, the score is 3
- If few requirements are matched, the score is 2
- If very few or no requirements are matched, the score is 1 or 0

6. Justify the match score:
<Justification>
- Provide specific examples from the requirements and resume to support why certain requirements
were considered matched or unmatched based on the criteria
- Explain your reasoning for the match score, referring to the proportion of requirements matched
and their importance for the criteria
</Justification>

7. Output your evaluation:
<MatchScore>SCORE</MatchScore>
JUSTIFICATION

Make sure to replace REQUIREMENTS with the actual requirements text, RESUME with the resume
text, SCORE with the 0-5 match score number, and JUSTIFICATION with your justification text
following the format specified.

If any of the inputs is missing or you cannot determine a score, simply output: "Unable to evaluate
due to missing information."

Before answering the question, please think about it step-by-step within <thinking></thinking> tags. Then, provide your final answer within <answer></answer> tags.

  """

In [15]:
def prepare_bedrock_request(messages, claude_config):
    return {
        'messages': messages,
        **claude_config,
    }
def extract_bedrock_response(response):
    body = json.loads(response['body'].read().decode('utf-8'))
    return body['content'][0]['text']

def pretty_print(message):
    print('\n\n'.join('\n'.join(line.strip() for line in re.findall(r'.{1,100}(?:\s+|$)', paragraph.strip('\n'))) for paragraph in re.split(r'\n\n+', message)))

## Simple test

In [136]:
%%time
messages = [
    {"role": "user", "content": [
        {"type": "text", "text": prompt},
        ]
    }
]

body = prepare_bedrock_request(messages, claude_config)
response = bedrock_client.invoke_model(modelId=model_id, body=json.dumps(body))
formated_response = extract_bedrock_response(response)

CPU times: user 4.19 ms, sys: 13 ms, total: 17.2 ms
Wall time: 37.9 s


In [112]:
pretty_print(formated_response)

<thinking>
Okay, let's go through this step-by-step:

1. Reading the requirements:
- Key technical skills: .NET Core 3.1, C# 8.0, SQL Server, HTML5, JavaScript, CI/CD, DevOps tools
like GitLab, Azure, Docker
- Methodologies: Agile, Scrum (based on mentions of product owner, backlogs, sprints)
- Experience level: Experienced, with requirements like "expérience reconnue en gestion de projets
SI", "expérience de la conduite du changement", ability to lead teams
- Main responsibilities: Project management, requirements gathering, functional specifications,
architecture design, testing, user support, process documentation

2. Reading the candidate's resume:
- Technical skills: .NET (WinDev, WebDev), SQL Server, Power BI, Oracle, Visual Basic, Unix, various
older languages/tools
- Experience: Over 30 years of experience across various roles like developer, tech lead, project
manager, product owner, director
- Certifications: One certification in C language under Unix
- Qualifications: 2 DUTs

## Tests with criteria

In [16]:
%%time
test_criteria = criteria_list_string[4]

criteria_prompt = get_criteria_prompt(test_criteria)
print(f"Criteria: {test_criteria}")
# print(f"Criteria prompt: {criteria_prompt}")
messages = [
    {"role": "user", "content": [
        {"type": "text", "text": criteria_prompt},
        ]
    }
]

body = prepare_bedrock_request(messages, claude_config)
response = bedrock_client.invoke_model(modelId=model_id, body=json.dumps(body))
formated_response = extract_bedrock_response(response)
pretty_print(formated_response)

Criteria: Connaissance du domaine fonctionnel (Facility management)
<thinking>
Étape 1 : Lire attentivement les exigences en gardant les critères à l'esprit
- Critère : Connaissance du domaine fonctionnel (Facility management)
- Les exigences mentionnent une mission liée au Facility Management (FM) pour valoriser les services
SI et apporter une forte valeur ajoutée aux clients sur les activités d'assistance à maîtrise
d'ouvrage sur les applications du pôle Services FM.
- Les tâches et responsabilités incluent la gestion de projet, l'assistance au pilotage budgétaire,
le reporting, la définition des besoins et spécifications fonctionnelles, l'architecture
fonctionnelle, les études de faisabilité, les recettes, le support aux utilisateurs, et la
formalisation des processus SI métiers de l'activité foncière et immobilière.
- Les compétences requises incluent l'expérience en AMOA SI dans le domaine immobilier, la gestion
de projets SI, les connaissances techniques SI (conception fonctionne

In [85]:
import xml.etree.ElementTree as ET
from xml.sax.saxutils import escape

def get_tag(xml_string, tag):
  # Find the start and end indices of the XML portion
  start_index = xml_string.find('<answer>')
  end_index = xml_string.find('</answer>') + len('</answer>')
  # Extract the XML portion
  xml_content = xml_string[start_index:end_index]
  # Parse the XML content
  print(xml_content)
  print(xml_content.replace("&", "&amp;"))
  root = ET.fromstring(xml_content.replace("&", "&amp;"))
  # Find the tag within the parsed XML content
  return root.find(tag).text

In [73]:
formated_response

"<thinking>\nÉtape 1 : Lire attentivement les exigences en gardant les critères à l'esprit\n- Critère : Connaissance du domaine fonctionnel (Facility management)\n- Les exigences mentionnent une mission liée au Facility Management (FM) pour valoriser les services SI et apporter une forte valeur ajoutée aux clients sur les activités d'assistance à maîtrise d'ouvrage sur les applications du pôle Services FM.\n- Les tâches et responsabilités incluent la gestion de projet, l'assistance au pilotage budgétaire, le reporting, la définition des besoins et spécifications fonctionnelles, l'architecture fonctionnelle, les études de faisabilité, les recettes, le support aux utilisateurs, et la formalisation des processus SI métiers de l'activité foncière et immobilière.\n- Les compétences requises incluent l'expérience en AMOA SI dans le domaine immobilier, la gestion de projets SI, les connaissances techniques SI (conception fonctionnelle d'application et urbanisation des échanges), la conduite d

In [79]:
score = get_tag(formated_response, "MatchScore")
justification = get_tag(formated_response, "Justification")

print(f"Match Score: {score}")
print(f"Justification: {justification}")

Match Score: 4
Justification: 
Le candidat possède une expérience très pertinente et approfondie dans le domaine fonctionnel du Facility Management, en particulier dans la gestion de projets liés aux ERP et SIRH pour les bailleurs sociaux. Ses responsabilités et tâches correspondent en grande partie aux exigences mentionnées, telles que la gestion de projet, la définition des besoins et spécifications fonctionnelles, l'architecture fonctionnelle, les études de faisabilité, le support aux utilisateurs, la formation, etc.

Cependant, certaines compétences techniques spécifiques mentionnées dans les exigences, comme .netCore 3.1, C# 8.0, HTML 5.0, JavaScript, CI, Gitlab, MS Azure, Dockers, ne sont pas directement mentionnées dans son CV. Bien que ces compétences ne soient pas essentielles pour le critère de connaissance du domaine fonctionnel, elles pourraient être utiles pour certaines tâches techniques liées au projet.

Dans l'ensemble, le candidat semble très bien correspondre aux exig

## Tests with all criteria

In [61]:
%%time


responses = []

for criteria in criteria_list_string:
    criteria_prompt = get_criteria_prompt(criteria)
    print(f"Criteria: {criteria}")
    # print(f"Criteria prompt: {criteria_prompt}")
    messages = [
        {"role": "user", "content": [
            {"type": "text", "text": criteria_prompt},
            ]
        }
    ]

    body = prepare_bedrock_request(messages, claude_config)
    response = bedrock_client.invoke_model(modelId=model_id, body=json.dumps(body))
    formated_response = extract_bedrock_response(response)
    responses.append(formated_response)


Criteria: Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)
Criteria: Nombre et pertinence des expériences similaires
Criteria: Compétences techniques
Criteria: Complexité des missions gérées
Criteria: Connaissance du domaine fonctionnel (Facility management)
CPU times: user 31 ms, sys: 46.7 ms, total: 77.7 ms
Wall time: 3min 37s


In [88]:
criteria_id = 0
for response in responses:
    score = get_tag(response, "MatchScore")
    justification = get_tag(response, "Justification")
    print(f"==== CRITERIA {criteria_id} ====")
    print(criteria_list_string[criteria_id])
    print(f"Match Score: {score}")
    print(f"Justification: {justification}")
    print("----------------------------------------")
    criteria_id += 1

<answer>
<MatchScore>4</MatchScore>
<Justification>
Le candidat correspond à la majorité des exigences clés liées aux critères de compréhension du besoin, d'outils et de méthodologie proposés. Avec son expérience de plus de 25 ans en tant que chef de projets R&D, directeur technique et DGA chez SCEPIA, il a démontré une solide expertise dans la gestion de projets SI, l'AMOA, le recueil et la rédaction d'expressions de besoins, les spécifications fonctionnelles, l'architecture fonctionnelle, les études d'opportunité et de faisabilité, la préparation et le suivi des recettes, le support aux utilisateurs, la formation des utilisateurs, la formalisation des processus SI métiers, ainsi que la capacité à piloter et animer des équipes pluridisciplinaires.

Bien que le candidat n'ait pas d'expérience mentionnée avec la stack technique spécifique demandée (.NET Core 3.1, C# 8.0, HTML 5.0, JavaScript, CI, GitLab, MS Azure, Docker), son expérience diversifiée avec d'autres technologies telles que

ParseError: syntax error: line 1, column 0 (<string>)